## Library preparation

In [843]:
%load_ext autoreload
%autoreload 2
%matplotlib inline #notebook

from IPython.display import Image, display
from JJcalc import *
from si_prefix import si_format
from io import StringIO 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import qcodes as qc
from qcodes.dataset.database import initialise_database
from qcodes.dataset.plotting import plot_by_id, get_data_by_id, plot_dataset

## To Do list
1. LS370 still crashes after around 7hrs continuous operation
1. Make new cables
    - Remake the I, V splitting Fisher probing line
    - Remake ISO, COM x2 Fisher-BNC cables
1. Maintain old sample holder
    - Fix pin 5b
1. Try to see if we can fix the wiring in the fridge. Currently ch 5ab open, ch 10ab short.
1. Make new sample holder

## Realtime data visualization